<a href="https://colab.research.google.com/github/Mustasheep/categorical_NLP/blob/main/pln_categorical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introdução

Este é um projeto de Processamento de Linguagem Natural (PLN) e tem o objetivo de prever textos de notícias (principalmente o título da matéria) e classificar em suas devidas categorias.

In [2]:
# Ocultando mensagens de erro
import warnings
warnings.filterwarnings('ignore')

## Importando bibliotecas
Para este projeto, irei utilizar um grande conjunto de bibliotecas:

**Manipulação de dados:**
  - `pandas` para manipular meus dados e preparar o DataFrame para PLN
      
**Pré-Processamento de texto:**
  - `re` para remoção de diversos caracteres
  - `spacy` para lematização de palavras da lingua portuguesa
  - `nltk` para remoção de stopwords, tokenização e lematização
  - `unidecode` para preservar acentuações da lingua portuguesa
      
**Vetorização e treinamento:**
 - Várias importações de pacotes do `sklearn` para diversos fins de vetorização, treinamento, modelo e métricas de avaliação.

**Dependências:**
 - Serão baixadas algumas dependências para serem aplicadas no pré-processamento     

In [3]:
!pip install unidecode > /dev/null
!python -m spacy download pt_core_news_sm > /dev/null

In [4]:
# Importando...
import pandas as pd

import re
import nltk
import spacy
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Dependências
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
nlp = spacy.load('pt_core_news_sm')

def pre_processamento_texto(texto, lemmatize=True):
    """
    A função irá remover tags HTML, acentos das palavras (permanecendo a letra),
    caracteres especiais, URLs, espaços em branco extras, capitalizar letras em
    minúsculas, remover números, tokenizar o texto, remover stopwords e lematizar.
    """


    texto = texto.strip()
    texto = texto.lower()
    texto = unidecode(texto)
    texto = re.sub(r'<.*?>', '', texto)
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)
    texto = re.sub(r'http\S+', '', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'\d+', '', texto)

    tokens = word_tokenize(texto)
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words] # Tokenização

    if lemmatize:
        tokens = [token.lemma_ for token in nlp(' '.join(tokens))] # Lematização

    return " ".join(tokens)

## Preparando dados

In [18]:
# O dataset foi preparado em outro projeto
df = pd.read_csv('/content/frases_categorias.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   texto      656 non-null    object
 1   categoria  656 non-null    object
dtypes: object(2)
memory usage: 10.4+ KB


,texto,categoria
0,Brasil vence a Argentina na final da Copa Amér...,Esporte
1,"Inflação desacelera em julho, aliviando a pres...",Economia
2,Cientistas descobrem nova espécie de peixe nas...,Ciência
3,Presidente anuncia pacote de medidas para esti...,Economia
4,Ator famoso revela segredo sobre seu novo filme.,Entretenimento


In [19]:
df['categoria'].value_counts()

,count
categoria,
Entretenimento,125
Política,103
Economia,87
Saúde,79
Tecnologia,72
Ciência,71
Esporte,60
Meio Ambiente,59


Existe um desbalanceamento nas quantidades das amostras dentro das categorias. Será necessário fazer um ajustamento na quantidade para balancear, mas provavelmente o uso de pesos irá resolver esse problema.

## Aplicando pré-processamento

In [22]:
# Aplicar pré-processamento
df['textos preprocessados'] = df['texto'].apply(pre_processamento_texto)
df.tail()

,texto,categoria,textos preprocessados
651,Descoberta de novas evidências sobre a vida em...,Ciência,descobrir novo evidencia sobre vida outro plan...
652,Empresa de tecnologia cria aplicativo que faci...,Tecnologia,empresa tecnologia criar aplicativo facilitar ...
653,Novo tratamento para doenças degenerativas apr...,Saúde,novo tratamento doenco degenerativo apresentar...
654,Exposição de quadros de pintores locais atrai ...,Entretenimento,exposicao quadro pintor local atrair morador v...
655,Aumento da criminalidade e da violência preocu...,Política,aumento criminalidade violencia preocupar mora...


## Vetorização e Treinamento
- Foi definido durante a vetorização um N-Grama `1,2` para preservar uma palavra, e em seguida, a mesma palavra com a seguinte. O objetivo disso é manter a sintaxe das frases e trazer mais sentido aos textos.

- Durante a divisão dos dados para o treinamento, eu fui ajustando os valores até obter um bom resultado das métricas, fixando o `teste_size=0.3`, random em `42` e espeficicando o `stratify` como a `categoria`.

- Como os valores estão desbalanceados, utilzei a regressão logística e atribui o peso `balanced`.

- Após observar métricas insatisfatórias, precisei implementar o Grid Search com validação cruzada. Isso ajudou muito a performance e eficácia do treinamento.

In [23]:
#------------------
# Vetorização TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(df['textos preprocessados'])
y = df['categoria']


#------------------
# Divisão dos dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#------------------
# Treinamento do modelo (Regressão Logística)
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

print("Shape X_train:", X_train.shape)
print("Shape X_test:", X_test.shape)

#------------------
# Grid Search com Validação Cruzada
param_grid = {'C': [0.1, 1, 10, 100],
              'solver': ['liblinear', 'lbfgs'],
              'penalty': ['l1', 'l2']}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', n_jobs=-1) # Usar todos os núcleos para grid search
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

#------------------
# Avaliação no conjunto de teste
y_pred = best_model.predict(X_test)
print(f'\nMelhores Parâmetros: {best_params}')
print(f"\n>> Acurácia: {(accuracy_score(y_test, y_pred) * 100):.1f}%")
print(f">> Precisão: {(precision_score(y_test, y_pred, average='weighted') * 100):.1f}%")
print(f">> Recall: {(recall_score(y_test, y_pred, average='weighted') * 100):.1f}%")
print(f">> F1-score: {(f1_score(y_test, y_pred, average='weighted') * 100):.1f}%")

Shape X_train: (459, 3780)
Shape X_test: (197, 3780)

Melhores Parâmetros: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}

>> Acurácia: 93.9%
>> Precisão: 94.0%
>> Recall: 93.9%
>> F1-score: 93.8%


In [24]:
print("\nRelatório de Classificação:\n\n", classification_report(y_test, y_pred, zero_division=0))


Relatório de Classificação:

                 precision    recall  f1-score   support

       Ciência       0.90      0.86      0.88        21
      Economia       1.00      0.96      0.98        26
Entretenimento       0.93      1.00      0.96        37
       Esporte       1.00      0.94      0.97        18
 Meio Ambiente       0.85      0.94      0.89        18
      Política       0.94      1.00      0.97        31
         Saúde       0.90      0.79      0.84        24
    Tecnologia       1.00      0.95      0.98        22

      accuracy                           0.94       197
     macro avg       0.94      0.93      0.93       197
  weighted avg       0.94      0.94      0.94       197



Com o `F1-score` em 94% será o suficiente para aplicar meu modelo com novos dados e testar minhas previsões.

## Aplicando modelo com novos dados

In [25]:
# Novas amostras para serem previstas
novos_coments = [
    "Bolsonaro aposta na influência de Trump para tentar reverter inelegibilidade.",
    "O mistério de por que a covid-19 parece estar se tornando menos letal",
    "Coldplay transmite maior show ao vivo na Índia pelo Disney+",
    "O valor do dolar está cada vez mais alto chegando a quase 7 reais."
]
novos_coments_preprocessados = [pre_processamento_texto(review) for review in novos_coments]
novos_coments_vetorizados = vectorizer.transform(novos_coments_preprocessados)
novos_coments_previsao = model.predict(novos_coments_vetorizados)

for review, prediction in zip(novos_coments, novos_coments_previsao):
    print(f"Comentário: {review} - Previsão: {prediction}\n")

Comentário: Bolsonaro aposta na influência de Trump para tentar reverter inelegibilidade. - Previsão: Política

Comentário: O mistério de por que a covid-19 parece estar se tornando menos letal - Previsão: Saúde

Comentário: Coldplay transmite maior show ao vivo na Índia pelo Disney+ - Previsão: Entretenimento

Comentário: O valor do dolar está cada vez mais alto chegando a quase 7 reais. - Previsão: Economia



## Conclusão


- Ao aplicar Oversampling, foi definido um balanceamento para que todas as categorias tivessem 125 amostras. Os resultados foram bons com métricas em torno de 98%. No entanto, o modelo ainda não conseguia avaliar novas amostragens com perfeição.

- Ao aplicar Ponderação de Classes, pude atribuir pesos diferentes às categorias usando parâmetros com sklearn na `LogisticRegression`. Os resultados foram bons com métricas em torno de 94%. E o modelo está prevendo bem as novas amostras, tornando mais ideal para este projeto.

- Foram adicionadas 4 amostras para previsão e o modelo conseguiu prever corretamente as frases
    - 1ª frase: previsão esperada `Política`
    - 2ª frase: previsão esperada `Saúde`
    - 3ª frase: previsão esperada `Entrenenimento`
    - 4ª frase: previsão esperada `Economia`

- Futuramente, pretendo encontrar ou acrescentar novas amostras para treinar o modelo com mais categorias.